In [60]:
# Detect metabolic models; alternatively, identify models that can be evaluated
# kegg.reaction & ec-code, something else? 
# In order to do that, need to collect all knowledge resources for reactions first
import itertools
import libsbml
import numpy as np
import os
import pickle
import pandas as pd
import sys
import matplotlib.pyplot as plt
%matplotlib inline  

PROJ_DIR = "/Users/woosubs/Desktop/AutomateAnnotation/AnnotationRecommender/"
MOD_DIR = os.path.join(PROJ_DIR, "annotation_recommender")
sys.path.append(MOD_DIR)

BIOMD_12 = 'BIOMD0000000012.xml'
BASE_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/'
DATA_DIR = os.path.join(BASE_DIR, "DATA")
ALGO_DIR = os.path.join(DATA_DIR, "algo")
CHEBI_DIR = os.path.join(DATA_DIR, "chebi")
RHEA_DIR = os.path.join(DATA_DIR, "rhea")
BIOMODEL_DIR = os.path.join(DATA_DIR, "biomodels/curated_biomodels_31mar2021")
BIGG_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/DATA/bigg'
ecoli_fpath = os.path.join(BIGG_DIR, "e_coli_core.xml")


from annotation_recommender import species_annotation as sa
from annotation_recommender import reaction_annotation as ra
from annotation_recommender import constants as cn
from annotation_recommender import iterator as it
from annotation_recommender import tools

# chebi to shortened formula
with open(os.path.join(CHEBI_DIR, 'chebi_shortened_formula_30apr2022.pickle'), 'rb') as f:
  ref_shortened_chebi_to_formula = pickle.load(f)
# shortened formula to chebi
with open(os.path.join(CHEBI_DIR, 'shortened_formula_to_chebis_20jul2022.pickle'), 'rb') as f:
  ref_shortened_formula_to_chebi = pickle.load(f)

with open(os.path.join(CHEBI_DIR, 'chebi_synonyms.pickle'), 'rb') as f:
  chebi_synonyms = pickle.load(f)
chebi_low_synonyms = dict()
for one_k in chebi_synonyms.keys():
  chebi_low_synonyms[one_k] = list(set([val.lower() for val in chebi_synonyms[one_k]]))

with open(os.path.join(RHEA_DIR, 'kegg2rhea_master.pickle'), 'rb') as handle:
  ref_kegg2rhea_master = pickle.load(handle)
with open(os.path.join(RHEA_DIR, 'kegg2rhea_bi.pickle'), 'rb') as handle:
  ref_kegg2rhea_bi = pickle.load(handle)

# load reference matrix
with open(os.path.join(ALGO_DIR, 'binary_ref_df.pickle'), 'rb') as handle:
    ref_mat = pickle.load(handle)
# check its shape
print(ref_mat.shape)

(13651, 3790)


In [2]:
# biomodels with kegg.reaction AND either CHEBI or kegg.metabolite:
biomd_keggs = pd.read_csv(
    os.path.join(BASE_DIR, 'reaction_recommender/reaction_recommender/notebook/biomodels_to_use.csv'),
    index_col=0)

In [3]:
# First, collect all identifiers for reaction annotations
all_biomds = [val for val in os.listdir(BIOMODEL_DIR) if val[-4:]=='.xml']
len(all_biomds)

1000

In [5]:
# one_biomd_fpath = os.path.join(BIOMODEL_DIR, one_biomd)
# species_an = sa.SpeciesAnnotation(libsbml_fpath=one_biomd_fpath)
# reaction_an = ra.ReactionAnnotation(libsbml_fpath=one_biomd_fpath)

In [6]:
# one_biomd = all_biomds[1]
all_qualifiers = set()
for one_biomd in all_biomds:
  one_fpath = os.path.join(BIOMODEL_DIR, one_biomd)
  reader = libsbml.SBMLReader()
  document = reader.readSBML(one_fpath)
  model = document.getModel()
  model_quals = []
  for one_r in model.getListOfReactions():
    one_list_quals = [val[0] for val in \
                      tools.getOntologyFromString(one_r.getAnnotationString())]
    model_quals = model_quals + one_list_quals
  all_qualifiers.update(model_quals)

In [7]:
all_qualifiers

{'bao',
 'biomodels.sbo',
 'doid',
 'ec-code',
 'efo',
 'go',
 'ido',
 'intact',
 'kegg.pathway',
 'kegg.reaction',
 'meddra',
 'ncbigene',
 'ncit',
 'obi',
 'obo.go',
 'obo.psi-mod',
 'omit',
 'pato',
 'psimi',
 'psimod',
 'pw',
 'reactome',
 'sbo',
 'subtiwiki',
 't3db',
 'uniprot'}

In [8]:
from bioservices import ChEBI, Rhea
r = Rhea()
df = r.search("")

In [9]:
df.head()

,Reaction identifier,Equation,ChEBI name,ChEBI identifier,EC number,Enzymes,PubMed,Cross-reference (EcoCyc),Cross-reference (MetaCyc),Cross-reference (KEGG),Cross-reference (Reactome)
0,RHEA:10000,H2O + pentanamide = NH4(+) + pentanoate,H2O;pentanamide;NH4(+);pentanoate,CHEBI:15377;CHEBI:16459;CHEBI:28938;CHEBI:31011,EC:3.5.1.50,0,NaN,NaN,MetaCyc:PENTANAMIDASE-RXN,KEGG:R02938,NaN
1,RHEA:10004,benzyl isothiocyanate = benzyl thiocyanate,benzyl isothiocyanate;benzyl thiocyanate,CHEBI:17484;CHEBI:16017,EC:5.99.1.1,0,13997458,NaN,MetaCyc:THIOCYANATE-ISOMERASE-RXN,KEGG:R04010,NaN
2,RHEA:10008,[protein]-dithiol + a hydroperoxide = [protein...,L-cysteine residue;a hydroperoxide;L-cystine r...,CHEBI:29950;CHEBI:35924;CHEBI:50058;CHEBI:3087...,NaN,8,10751410;12033427;12517450;9587003,NaN,NaN,NaN,NaN
3,RHEA:10012,(R)-6-hydroxynicotine + H2O + O2 = 6-hydroxyps...,(R)-6-hydroxynicotine;H2O;O2;6-hydroxypseudoox...,CHEBI:58413;CHEBI:15377;CHEBI:15379;CHEBI:5868...,EC:1.5.3.6,1,16095622;2680607,NaN,MetaCyc:R-6-HYDROXYNICOTINE-OXIDASE-RXN,KEGG:R07170,NaN
4,RHEA:10016,H2O + O-sinapoylcholine = choline + E-sinapate...,H2O;O-sinapoylcholine;choline;E-sinapate;H(+),CHEBI:15377;CHEBI:16353;CHEBI:15354;CHEBI:3002...,EC:3.1.1.49,1,18036206,NaN,MetaCyc:SINAPINE-ESTERASE-RXN,KEGG:R02381,NaN


In [45]:
ec2rhea_master = dict()
for idx in df.index:
  one_row = df.loc[idx, :]
  ec_val = one_row['EC number']
  # float will indicate entry is NaN
  if not isinstance(ec_val, float):
    ecs = ec_val.split(';')
    for one_ec in ecs:
      if one_ec in ec2rhea_master.keys():
        ec2rhea_master[one_ec[3:]] = ec2rhea_master[one_ec[3:]] + [one_row['Reaction identifier']]
      else:
        ec2rhea_master[one_ec[3:]] = [one_row['Reaction identifier']]

In [47]:
ec2rhea_master['3.5.1.50']

['RHEA:10000']

In [48]:
kegg2rhea_master = dict()
for idx in df.index:
  one_row = df.loc[idx, :]
  kegg_val = one_row['Cross-reference (KEGG)']
  # float will indicate entry is NaN
  if not isinstance(kegg_val, float):
    keggs = kegg_val.split(';')
    for one_kegg in keggs:
      if one_kegg in kegg2rhea_master.keys():
        kegg2rhea_master[one_kegg[5:]] = kegg2rhea_master[one_kegg[5:]] + [one_row['Reaction identifier']]
      else:
        kegg2rhea_master[one_kegg[5:]] = [one_row['Reaction identifier']]

In [49]:
kegg2rhea_master['R02938']

['RHEA:10000']

In [14]:
# below shows that currently all values are mapped into BI directions
ref_mat.head()

,C10N2O11P,C45CoN4O14,C17N4O10P,C45O,C20O2,C12N5O13P2,C30BrClN6O9,C23FN7O17P3S,C19N3O7,C35FeN4O6,...,C22NO,C15O6,C11N2O4P,C41N9O28P2,C57O5,C10N2O3,C17N5O17P3R,C32N5O8P,C40N9O13,C22N2O2
RHEA:10003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
RHEA:10007,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
RHEA:10011,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
RHEA:10015,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
RHEA:10019,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
# But anyways, now we can figure out models (and reactions) that have either EC- or KEGG- number 
# that can be mapped into RHEA terms

# Next, find models and associated reactions with qualifiers, esp. that can be mapped to rhea; 
# Of course, find information to map into rhea

In [33]:
# def formatECNumber(inp_str):
#   """
#   Format EC number so that
#   the number becomes 'EC:XX.XX.XX.XX'
  
#   Parameters
#   ----------
#   inp_str: str
  
#   Returns
#   -------
#   res_str: str
#   """
#   if inp_str[:3]=='EC:':
#     res_str = inp_str
#   else:
#     res_str = 'EC:' + inp_str
#   return res_str

In [53]:
eckegg_models = dict()
for one_biomd in all_biomds:
  one_fpath = os.path.join(BIOMODEL_DIR, one_biomd)
  reader = libsbml.SBMLReader()
  document = reader.readSBML(one_fpath)
  model = document.getModel()
  reaction_info = dict()
  for one_r in model.getListOfReactions():
    ec_anots = [val[1] for val in \
                tools.getOntologyFromString(one_r.getAnnotationString()) \
                if val[0] == 'ec-code']
    kegg_anots = [val[1] for val in \
                tools.getOntologyFromString(one_r.getAnnotationString()) \
                if val[0] == 'kegg.reaction']
    # filter to get 'evaluatable' identifiers
    ec_anots_filt = [val for val in ec_anots if val in ec2rhea_master.keys()]
    kegg_anots_filt = [val for val in kegg_anots if val in kegg2rhea_master.keys()]
    #
    eval_qualifiers = dict()
    if len(ec_anots_filt)>0:
      eval_qualifiers['ec'] = ec_anots_filt
    if len(kegg_anots_filt):
      eval_qualifiers['kegg'] = kegg_anots_filt
    #
    if len(eval_qualifiers.keys()) > 0:
      reaction_info[one_r.getId()] = eval_qualifiers
  # models collected only if model has at least one (EC or KEGG) annotation
  if len(reaction_info.keys()) > 0:
    eckegg_models[one_biomd] = reaction_info

#
print("Total nubmer of models: %d" % len(eckegg_models.keys()))

Total nubmer of models: 131


In [64]:
eckegg2rhea_models = dict()
for one_md in eckegg_models.keys():
  one_model_itm = eckegg_models[one_md]
  model_anot = dict()
  for one_r in one_model_itm.keys():
    reaction_anot = []
    one_reaction_itm = one_model_itm[one_r]
    if 'ec' in one_reaction_itm.keys():
      one_itm = one_reaction_itm['ec']
    
#     list(itertools.chain(*list2d))
      reaction_anot = reaction_anot + list(itertools.chain(*[ec2rhea_master[val] for val in one_itm]))
    if 'kegg' in one_reaction_itm.keys():
      one_itm = one_reaction_itm['kegg']
      reaction_anot = reaction_anot + list(itertools.chain(*[kegg2rhea_master[val] for val in one_itm]))
    if len(reaction_anot) > 0:
      model_anot[one_r] = list(set(reaction_anot))
  eckegg2rhea_models[one_md] = model_anot

In [66]:
len(eckegg2rhea_models)

131

In [67]:
# save results
with open('eckegg2rhea.pickle', 'wb') as handle:
    pickle.dump(eckegg2rhea_models, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Now, for species? 

In [23]:
# one_biomd = all_biomds[1]
all_qualifiers = set()
for one_biomd in all_biomds:
  one_fpath = os.path.join(BIOMODEL_DIR, one_biomd)
  reader = libsbml.SBMLReader()
  document = reader.readSBML(one_fpath)
  model = document.getModel()
  model_quals = []
  for one_s in model.getListOfSpecies():
    one_list_quals = [val[0] for val in \
                      tools.getOntologyFromString(one_s.getAnnotationString())]
    model_quals = model_quals + one_list_quals
  all_qualifiers.update(model_quals)

In [25]:
print(all_qualifiers)

{'tcdb', 'kegg.orthology', 'cco', 'ncbiprotein', 'ncit', 'biomodels.sbo', 'pubchem.substance', 'kegg.drug', 'bto', 'go', 'pirsf', 'ec-code', 'omit', 'unipathway.compound', 'bao', 'obo.fma', 'sbo', 'cl', 'brenda', 'lipidmaps', 'chebi', 'fma', 'pubchem.compound', 'chembl.compound', 'omim', 'taxonomy', 'psimi', 'obi', '3dmet', 'uniprot.isoform', 'ncbigene', 'interpro', 'obo.go', 'ensembl', 'bind', 'efo', 'mirbase', 'cas', 'kegg.reaction', 'so', 'uniprot', 'mirbase.mature', 'sgd', 'pato', 'ncim', 'obo.pato', 'pr', 'kegg.genes', 'psimod', 'pw', 'ido', 'kegg.compound', 'obo.chebi', 'orphanet', 'reactome'}


In [29]:
spec_qualifiers = ['kegg.compound', 'chebi', 'obo.chebi']
chebikegg_models = dict()
for one_biomd in all_biomds:
  one_fpath = os.path.join(BIOMODEL_DIR, one_biomd)
  reader = libsbml.SBMLReader()
  document = reader.readSBML(one_fpath)
  model = document.getModel()
  species_info = dict()
  for one_s in model.getListOfSpecies():
    chebikegg_anots = [val[1] for val in \
                       tools.getOntologyFromString(one_s.getAnnotationString()) \
                       if val[0] in spec_qualifiers]
    if len(chebikegg_anots) > 0:
      species_info[one_s.getId()] = chebikegg_anots
  if len(species_info.keys()) > 0:
    chebikegg_models[one_biomd] = species_info
print("Total nubmer of models: %d" % len(chebikegg_models.keys()))

Total nubmer of models: 445


In [31]:
len(set(list(eckegg_models.keys()) + list(chebikegg_models.keys())))

484

In [75]:
# Collect models again, this time with only species with CHEBI terms
spec_qualifiers = ['chebi', 'obo.chebi']
chebispecies_models = dict()
for one_biomd in all_biomds:
  one_fpath = os.path.join(BIOMODEL_DIR, one_biomd)
  reader = libsbml.SBMLReader()
  document = reader.readSBML(one_fpath)
  model = document.getModel()
  species_info = dict()
  for one_s in model.getListOfSpecies():
    chebi_anots = [val[1] for val in \
                   tools.getOntologyFromString(one_s.getAnnotationString()) \
                   if val[0] in spec_qualifiers]
    # Filter to get only chebi terms included in the CHEBI-formula dictionary
    filt_chebi_anots = [val for val in chebi_anots if val in ref_shortened_chebi_to_formula.keys()]
    if len(filt_chebi_anots) > 0:
      species_info[one_s.getId()] = filt_chebi_anots
  if len(species_info.keys()) > 0:
    chebispecies_models[one_biomd] = species_info
print("Total nubmer of models: %d" % len(chebispecies_models.keys()))

Total nubmer of models: 306


In [76]:
# save results
with open('chebi_models.pickle', 'wb') as handle:
    pickle.dump(chebispecies_models, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [81]:
len(set(chebispecies_models.keys()).intersection(list(eckegg_models.keys())))

83